# Advanced Analytics(AA) COE OCR and Data Extraction Process Overview


### Stage 1: Preprocess ,  OCR, and Transform Original Documents
    #### 1. Identify if PDF is searchable or unsearchable 
    #### 2. Repair , Clean and enhance original pdf(with image) 
    #### 3. Use LSTMs + OSD configuration from Tesseract to OCR PDF and Create searchable document 
    #### 4. Convert and Save searchable PDF into Formatted Text file preserving all spacing and formatting of original pdf 
    #### 5. Load Formated txt file and store each row individually into a list 

### Stage 2: Attribute Extraction 
    #### 1. Create Dictonary of Reg-ex patterns to identify attribute and table locations
    #### 2. Search rows in doc for a match to a reg-ex pattern associated with attribute in question (row, startpos, endpos)
    #### 3. Parse attribute value from text based on loc relationship with attribute header and apply Checks to filter out FPs
            # 1. Check text to right of header (first word > 5 characters, first 2 words are not text, text contains num) 
            # 2. if no text found to right , check text BELOW attribute header using header loc as starting pos for next line 
    #### 4. Apply check to ensure correct attribute extracted (check:  1 group of text with  numbers, chars,  special chars
    
### Stage 3: Table Extraction 
    #### 1. Find starting row of table using regex patterns stored in Dictonary that indicate table start 
    #### 2. Find ending location of table using patterns associateed with end of table for a particular company 
    #### 3. Extract information between start and end row of table using delimiter(double space, works 80% of time)
    #### 3. For ~100% Accuracy, simply define character ranges that are associated with the table for each comapany
        
##### Date Creation: 9/21
##### Last Edited: 10/5
##### Authors: Zack Carideo + Ran Zhou    

In [1]:
#Import Modules
import os 
import re 
from collections import OrderedDict
from operator import itemgetter
#from StringIO import StringIO
import stat
import sys, getopt
from subprocess import Popen, PIPE
import subprocess
import pdftotext
from re import finditer
import fileinput

## 1.Identify Unserachable PDFS to Be Converted to Searchable 

In [4]:
#location of all POs , Results, and Cropped POs
path = "//home//docadmin//python//ocr//samples//PDFs//" #"//home//kgplatformuser//python//ocr//samples//PDFs//"
outpath =  "//home//docadmin//python//ocr//samples//SearchablePDF_Sandwiches//"

#identify searchable vs non searchable pdfs 
#note: pdfs must be stored in a directory with no other folders 
searchable , unsearchable = IsPDFSearchable(path)

#create paths to searchable pdfs (only pull pack the ones that have been cropped and extra pages removed)
UnSearchablePDFPaths = [path + fn for fn in unsearchable if fn.endswith('.pdf')]     
SearchablePDFPaths = [path + fn for fn in searchable if fn.endswith('.pdf')]
print("Searchable PDF's: " ,SearchablePDFPaths,"\n")
print("UnSearchable PDF's: ",UnSearchablePDFPaths)


Searchable PDF's:  ['//home//docadmin//python//ocr//samples//PDFs//PO14_GM.pdf', '//home//docadmin//python//ocr//samples//PDFs//PO11.pdf', '//home//docadmin//python//ocr//samples//PDFs//PO16.pdf', '//home//docadmin//python//ocr//samples//PDFs//PO15_GM.pdf', '//home//docadmin//python//ocr//samples//PDFs//PO9_GM.pdf', '//home//docadmin//python//ocr//samples//PDFs//PO5.pdf', '//home//docadmin//python//ocr//samples//PDFs//PO6_GM.pdf', '//home//docadmin//python//ocr//samples//PDFs//PO17_GM.pdf', '//home//docadmin//python//ocr//samples//PDFs//PO12.pdf', '//home//docadmin//python//ocr//samples//PDFs//PO3.pdf', '//home//docadmin//python//ocr//samples//PDFs//PO8.pdf'] 

UnSearchable PDF's:  ['//home//docadmin//python//ocr//samples//PDFs//PO1.pdf', '//home//docadmin//python//ocr//samples//PDFs//PO10.pdf', '//home//docadmin//python//ocr//samples//PDFs//PO2.pdf', '//home//docadmin//python//ocr//samples//PDFs//PO7.pdf', '//home//docadmin//python//ocr//samples//PDFs//PO13_Email.pdf']


## 2. Repair, pre-process and configure original pdf into searchable pdf using Tesseract 

#### ML Based OCR: LSTM Neural Networks(Text recognition) + OSD Tesseract(Page segmentation and layout analysis) 

In [5]:
#generate pdf-a (default output for ocrmypdf)
SearchablePDFOutpaths= []
for i in range(len(UnSearchablePDFPaths)):
    pdf_output_path = outpath + str(UnSearchablePDFPaths[i].rsplit("//", 1)[1]).replace(".pdf","")+"_V2searchable.pdf"
    os.system("ocrmypdf --output-type pdf --deskew --clean --rotate-pages --clean-final "+ UnSearchablePDFPaths[i]+" "+ pdf_output_path )
    SearchablePDFOutpaths.append(pdf_output_path)          
    print(pdf_output_path)        
    
#Input / Output Locations
#Original PDF Files://home//docadmin//python//ocr//samples//PDFs//
#Final Sandwich PDF Files://home//docadmin//python//ocr//samples//SearchablePDF_Sandwiches//
#Formatted PO TXT Files: //home//docadmin//python//ocr//samples//Converted_txtfiles//

//home//docadmin//python//ocr//samples//SearchablePDF_Sandwiches//PO1_V2searchable.pdf
//home//docadmin//python//ocr//samples//SearchablePDF_Sandwiches//PO10_V2searchable.pdf
//home//docadmin//python//ocr//samples//SearchablePDF_Sandwiches//PO2_V2searchable.pdf
//home//docadmin//python//ocr//samples//SearchablePDF_Sandwiches//PO7_V2searchable.pdf
//home//docadmin//python//ocr//samples//SearchablePDF_Sandwiches//PO13_Email_V2searchable.pdf


## 3. Use PDF2Text to Generate Layout specific text file associated with Each 
## Max Note: This is last step that would need to be wrapped into closed source function

In [6]:
#Path tied to converted ocrmypdf pdf files 
pdf_path = "//home//docadmin//python//ocr//samples//SearchablePDF_Sandwiches//"

#Destination path for formatted text output
txt_path = "//home//docadmin//python//ocr//samples//Converted_txtfiles//"

#pdf file names found in the pdf_path location
files = getfiles(pdf_path, ".pdf")

#empty list to hold paths to txt files 
pdftextpaths=[]

#loop through each OCRMYPDF converted pdfs, and convert to text with layout of original pdf maintained as text 
for i in range(len(files)):
    txtpath = txt_path + files[i].replace("_V2searchable.pdf",".txt")
    pdfpath = pdf_path + files[i]
    
    #append all text paths
    pdftextpaths.append(txtpath)
    
    #convert all pdfs to text and preserve layout 
    os.system('pdftotext -layout '+pdfpath+' '+txtpath)
    
#loop through all searchable pdfs and convert to text with layout of original pdf maintained as text 
for i in range(len(SearchablePDFPaths)):
    txtpath = txt_path + str(SearchablePDFPaths[i].rsplit("//", 1)[1]).replace(".pdf","")+".txt"
    os.system('pdftotext -layout '+SearchablePDFPaths[i]+' '+txtpath)
    pdftextpaths.append(txtpath)

## 4.Read in Text File to be Processed and split it into individual lines

In [27]:
#text files 
files = getfiles(txt_path, ".txt")

#file number(variable or needs to be an iterable object)
filenum =14

#Open file and split lines into list based on new line indicator
#https://stackoverflow.com/questions/34399172/why-does-my-python-code-print-the-extra-characters-ï-when-reading-from-a-tex
#f =io.open("//home//docadmin//python//ocr//samples//SearchablePDF_Sandwiches//PO1.txt", encoding = "utf-8-sig")
#right:1 , below: 7 , table1: 5, table2: 14
f =open(txt_path+files[filenum],encoding = "utf_8_sig")
a = f.read()
b = a.split("\n")

#remove all completely white lines (helps postprocessing and identifying values that are below associated attribute)
c = [x for x in b if x]

#view output
c[:35]

['                                                                              Purchase Order',
 '                                                                              Date                         12/9/2015',
 '                                                                              PO #                         409774',
 'GRAY MATTER SYSTEMS, LLC',
 '                                                                              Currency                     US Dollar',
 '100 GLOBAL VIEW DRIVE',
 'WARRENDALE PA 15086                                                           Class                        GMS : Central US',
 'United States                                                                 Terms                        NET 30 DAYS',
 '(412)741-2410 Fax(412)741-2697                                                End User',
 'www.graymattersystems.com',
 'Vendor                                                                                         Ship To',
 'GE Inte

# BEGIN Attribute and Table Extraction 

## 5.Create Dictonary to hold all patterns used to identify attribute names 

In [28]:
#create empty dictonary to hold key value pairs(attribute : regex)
#Note:holds all attributes and the regex used to find each
LookupTable = {}

#dictonary key value pairs (Attribute:Regex)
LookupTable['PO NUMBER']=re.compile(r'Order Number:|CHANGE PURCHASE ORDER:|\s{4}(P/O NUMBER)|P.O. NO.:|P.O. #:|PO #|PO Number|PO Number:| Purchase Order No:(#)?|PURCHASE ORDER(#)?')

#dictonary key value pair to identify start line of table
LookupTable['TABLE']=re.compile(r'Item #+( +PARTICULAR)(.*)|\s+ITEM+( +P. REQ.)(.*)|\s+ITEM+( +QTY)+( +UM)(.*)|^Item+( +Material/Service)+( Description)(.*)|\s+Line+( +Quantity)+( +Item)(.*)|\s+LN+( +ORDER)+( +QTY)(.*)|^Item+( +Quantity)(.*)|^Line+( +Item)(.*)|^Item Nr+( +Description)(.*)')


#ZJC 10/10: May need multiple dictonaries(or dictonary of dictonaries), 1 to hold key/value attribute search rules, 1 to hold rules to validate results 

#dictonary key value pair used as control rule to ensure the PO number returned is not 2 full non integer words
LookupTable['MultipleWordTest']=re.compile(r'^([a-zA-Z]+\s){2}')

#dictonarykey value pairs used to identify the end line of a table based on end of table identifiers 
#Note: This is comapny specific, i have shown for Grey Matters Inc below 
LookupTable['GREYTABLEEND1']=re.compile(r'(?<!\S)Subtotal(?!\S)')
LookupTable['GREYTABLEEND2']=re.compile(r'(^|\s{4,})Total(\s+|$)') #at least 4 spaces before the word being searched for


#look whats in the dictonary 
list(LookupTable.keys())
for x in LookupTable.keys(): 
    print(x)
    
#reference the dictonary element for [PO NUMBER]
print(LookupTable['PO NUMBER'])

GREYTABLEEND1
PO NUMBER
TABLE
GREYTABLEEND2
MultipleWordTest
re.compile('Order Number:|CHANGE PURCHASE ORDER:|\\s{4}(P/O NUMBER)|P.O. NO.:|P.O. #:|PO #|PO Number|PO Number:| Purchase Order No:(#)?|PURCHASE ORDER(#)?')


In [11]:
#search through each line of document and identify where matches for PO number have been found       
for i in range(len(c)):
    print(str(i),re.search(LookupTable['PO NUMBER'],c[i]))

0 None
1 None
2 <_sre.SRE_Match object; span=(78, 82), match='PO #'>
3 None
4 None
5 None
6 None
7 None
8 None
9 None
10 None
11 None
12 None
13 None
14 None
15 None
16 None
17 None
18 None
19 None
20 None
21 None
22 None
23 None
24 None
25 None
26 None


## 6.Store match information for the attribute in question into a new dictonary 

In [22]:
#NOTE: TEST contains all matches for the element in question
#identify start and end locaiton of string found for each line in po
test ={}
list1 = []
#for every line in the PO 
for i in range(len(c)):
   
    # apply regex pattern associated with attribute in question 
        # and for each pattern match found in line i generate locational information 
            # to use to parse / find attribute value 
    for match in finditer(LookupTable['PO NUMBER'], c[i]):
     
        #output nested list with tuple of locational information about the match 
        list1.append(([str(i)],[match.span()], [match.group()]))#create a 3 element tuple with the information on locaiton of string

#assign dictonary values for "PO Number" KEY from the list above 
##THIS NEEDS TO CHANGE SOMEHOW SO INSTEAD OF STATIC '1' we just reference 'PO NUMBER' (this wont be dynamic though)
#test[list(LookupTable.keys())[1]] = list1 #change 1 to an iterable object to make dynamic
test['PO NUMBER'] = list1 

#dictonary with mutltiple values for each key value is equivalent to nested list in R
for i in test.keys():
    for j in test[i]:
        print(j)

(['0'], [(184, 193)], ['PO Number'])
(['44'], [(185, 194)], ['PO Number'])
(['73'], [(0, 14)], ['PURCHASE ORDER'])


'PO NUMBER'

## 7.Parse Attribute(PO#) from text based on  locational relationship with the attribute name and apply control check to ensure text is valid entry for attribute being extracted 

In [25]:
#Create empty string to hold result
EndRes = ""

#Create Empty list to hold Desriptive sttatistics about line in which matches were found 
matchdetails = []

#look through each match found in prior step and determine if the expected attribute value is to the right or below the attribute header/name
for i in test.keys():
    for j in test[i]:
        
        #Aggregated details on matched strings  
        matchdetails.append(
                               ( int(j[0][0])                             #Line in text inwhich the match was found 
                               , wordCount(stripwhite(c[int(j[0][0])]))   #Totalwords in line string was found
                               , len(c[int(j[0][0])])                     #Length of characters in the line the match was found 
                               , j[1][0]                                  #Start and end position of the matched pattern found 
                               , j[2][0]                                  #Actual value the pattern matched on 
                               ))

        #processing each match to find true result
        if list(test.keys())[0] == 'PO NUMBER':
            #string being processed
            string = c[int(j[0][0])]
            #print(c[int(j[0][0])+1])
                
            #First Check information to the right of Attribute Name
            #all text to the right of the string matched on
            ToRight_Orig = string[string.find(j[2][0])+len(j[2][0]):len(string)] #starting position of match + num characters associated with attribute being searched for : eol
            ToRight_Strip = string[string.find(j[2][0])+len(j[2][0]):len(string)].strip()
            #print("TEXT",ToRight_Orig) \ print(len(ToRight_Strip))
      
            #ensures no null values are passed or sequences of words aer passed to the PO Number Match Logic 
             #1.length of entire string >5
             #2.length of the first word in string > 5 (this will be the value we pull for PO)
             #3.String does not contain more than 2 consequtive words at bengingin of text to right of attribute found
            if len(ToRight_Strip)> 5 \
            and len(ToRight_Strip.split()[0])>5 \
            and re.search(LookupTable['MultipleWordTest'],ToRight_Strip) is None:  
                
                #next word in string after the matched word 
                nextword = ToRight_Strip.split()[0]
                print(nextword)
                
                #for each match apply the following regex check: 
                #1.find an optional letter with anything before it
                #2.a number with anything before it
                #3.and an optional special character with anything before it
                for match in finditer(re.compile(r'(.*[a-zA-Z])?(.*[0-9])(.*[!-@#$%&*])?'), nextword):
                    print(match.group())
                    EndRes = match.group() #if result is found at match i , this is populated and prevents further matches from being considered
         
    #if you cannot find any matches when looking to the right , look towards the next 
    if EndRes =="": 
        itr=1 #initial value for number of lines to move down from original Attribute found 
        for j in test[i]:
            #print(j)

            #continue to move down the lines 1 at a time until the length of the orig + itr line is similar to the length 
            #of the line the attribute was found at. Becuase our text layout is preserved, the line with informaiton of PO should 
            #be of simlar length to the line the attribute header was pulled from 
            #NOTE: When comparing line lengths, we do not use the striped line, we want to preserve all leading and trailing spaces
            if len(c[int(j[0][0])+1]) < round(len(c[int(j[0][0])])-(len(c[int(j[0][0])])*.05),0):
                while len(c[int(j[0][0])+itr]) < round(len(c[int(j[0][0])])-(len(c[int(j[0][0])])*.05),0):
                
                    itr = itr + 1
                    nextstring = c[int(j[0][0])+itr]
                    print('A',nextstring)
                    #print(len(c[int(j[0][0])+itr]))
                    #print( len(c[int(j[0][0])])-len(c[int(j[0][0])])*.1)

                #else:#if the next line is of similar length to the line above, u dont need to use while loop, just take row i+1
                #    nextstring = c[int(j[0][0])+itr]
                    #print('B',nextstring)
            else: 
                #RISK: NEED CONTROL TO PULL BACK NULL OR SOMETHING IF THE ABOVE CONDITIONS DONT HOLD
                nextstring = c[int(j[0][0])+itr]
                
            nextstring_strip = nextstring.strip()
            #print(len(nextstring),int(j[1][0][0]))
            print(nextstring)
            
            #length of next string 
            Length = len(nextstring)
            itr=2 #initialize i
             
            #if there are 0 words picked up in the text below(if its just blank space below)
            if wordCount(stripwhite(nextstring[int(j[1][0][0])-itr:j[1][0][1]+itr])) < 1:
                
                #Continue to expand the space on the line considered until there is at least 1 word(EXPAND UNIFORMLY ON EACH SIDE)
                while wordCount(stripwhite(nextstring[int(j[1][0][0])-itr:j[1][0][1]+itr])) < 1:
                    if itr==2: #if first iteration
                        Below_Text = nextstring[int(j[1][0][0])-itr:j[1][0][1]+itr] #pull in positioin of header +/- 2 char for first try
                        itr= int(round(Length / 20,0)) #set i for start of next loops
                        #print(Below_Text)
                    #expand the region considered out by a factor of 1/20th the total characters in the line until at least 2 words are present 
                    Below_Text = nextstring[int(j[1][0][0])-itr:j[1][0][1]+itr]
                    itr= itr + int(round(Length/20,0))
            else: 
                 for i in range(0,20,2):
                        if wordCount(stripwhite(nextstring[int(j[1][0][0])-i:j[1][0][1]+i])) <= 1:
                            Below_Text = nextstring[int(j[1][0][0])-i:j[1][0][1]+i] #pull in positioin of header +/- 2 char for first try
                            print(Below_Text)
                        
                
            #Loop through all matches and extract the associated PO Number 
            for match in finditer(re.compile(r'(.*[a-zA-Z])?(.*[0-9])(.*[!-@#$%&*])?'), Below_Text):
                    print(match.group())
                    EndRes = stripwhite(match.group())

#print the end result
EndRes



A                                       Col.INDUSTRIAL VALLEJO, CP 02300
A                                                                                                                                                                                     4506067861
                                                                                                                                                                                    4506067861
067861
06067861
4506067861
  4506067861
    4506067861
      4506067861
        4506067861
          4506067861
            4506067861
              4506067861
              4506067861
A                                                                                                                                                                                     4506067861
                                                                                                                                                                     

'4506067861'

## 8.Company Specific Table Parsing 
### Script Built For: Grey Matters Inc.

##### Note: I duplicate some key queries from above, once script is enclosed in function we will need to have it iterate through each variable using the same script, its a waste to repeat yourself! ####

### 1.Find Start Location / Row of Table 
##### Use regex rules searching for presence of column names associated with the table

In [29]:
#NOTE: TEST contains all matches for the element in question
#identify start and end locaiton of string found for each line in po 
test ={}
list1 = []

#for every line in the PO 
    # apply regex pattern associated with attribute in question 
    # and for each pattern match found in line i generate locational information 
    # to use to parse / find attribute value 
for i in range(len(c)):
       
    #check for match in each line of the rule associated with "TABLE" key
    for match in finditer(LookupTable['TABLE'], c[i]):
        
        #output nested list with tuple of locational information about the match 
        #create a 3 element tuple(line #, where match was found, what match was) with the information on locaiton of string
        list1.append(([str(i)],[match.span()], [match.group()]))

#assign dictonary values for "TABLE" KEY from results of lookuptable dic stored in the list above to a new dict with same Key ('TABLE') holding results of looping through each line in doc and applying rule from fist dictonary 
#
#0 = PO , 1=Mulitwordtest ,2 = Table, 3 = tbleend1 , 4 = tableend2
#

#we refecrence list(LookupTable.keys())[2] simply to ensure we use the same naming convention in the new list
test[list(LookupTable.keys())[2]] = list1

print(test)

{'TABLE': [(['14'], [(0, 122)], ['Item                   Quantity      Description                                                 Rate               Amount'])]}


### 2.Identify Start / End location of each header in the table

##### Note: Table Column widths are consistent, so we can pre-define the column width associated with each header to improve accuracy(at the cost of flexibiilty) if needed 

In [30]:
import operator
for i in test.keys():
    for j in test[i]:
        
        #header line (actual string of text where match was found)
        line = c[int(j[0][0])]
        
        #split line into groups of words (seperated by like 4 or 5 spaces) 
        split = line.split('   ')
        
        #remove blank elemnts from list
        split = list(filter(None, split))
        
        #loop through each header value and identify the start and end location of the string 
        StartLoc = []
        for z in range(len(split)):
            
            #header word
            word = split[z]
            
            #start and end loc of each word in the line being parsed
            StartLoc.append([stripwhite(str(word)), int(j[0][0]) ,line.find(str(word)), line.find(str(word))+len(word)])    
print(StartLoc)                      

[['Item', 14, 0, 4], ['Quantity', 14, 22, 31], ['Description', 14, 37, 48], ['Rate', 14, 96, 101], ['Amount', 14, 116, 122]]


### 3.Identify the ending location of the table 
##### Note: Greymatter tables always end after Total or Subtotal line . First we check subtotal because that will always appear before total, if subtotal is not found we revert to finding "Total"

In [38]:
#Find Ending location of Table (GreyMatter: either line above subtotal or line above total)
list2 = []
test2 = {}

#Indicators 
Grey1 = None
Grey2 = None 

for i in range(len(c)):
    
    #First look for SUBTOTAL becuase it will always be first(if its there)
    for match in finditer(LookupTable['GREYTABLEEND1'], c[i]): 
        if match is None: #exit loop if no match 
            break    
            
        #output nested list with tuple of locational information about the match 
        list2.append(([str(i)],[match.span()], [match.group()]))#create a 3 element tuple with the information on locaiton of string
        
        #indicator (indicates subtotal found)
        Grey1 = 1
        
for i in range(len(c)): 
    
    #is None: #If you dont find subtotal look for TOTAL 
    if list2 ==[]:
        for match in finditer(LookupTable['GREYTABLEEND2'], c[i]):
            #output nested list with tuple of locational information about the match 
            list2.append(([str(i)],[match.span()], [match.group()]))#create a 3 element tuple with the information on locaiton of string
            #indicator 
            Grey2 = 1
        
        
#assign obect to dictonary? 
if Grey1==1:
    #test2[list(LookupTable.keys())[3]] = list2
    test2['GREYTABLEEND1'] = list2
    
    endvar = stripwhite(test2[list(LookupTable.keys())[3]][0][2][0])
    print(endvar)
    print("SubTotal Found")
    
if Grey2==1: 
    test2['GREYTABLEEND2'] = list2
    endvar = stripwhite(test2[list(LookupTable.keys())[4]][0][2][0])
    print(endvar)
    print("Total Found")
    
#print matchesi
for i in test2.keys():
    for j in test2[i]:
        print(j)
        
#Begining / Ending Line of table 
TableEnd = int(list2[0][0][0])
TableBeg = int(list1[0][0][0])
TableRange = TableEnd - TableBeg

#total rows in table 
TableRange

KeyError: 'MultipleWordTest'

In [37]:
list(LookupTable.keys())[0]

'GREYTABLEEND1'

### 4.Extract Table information using pandas 

In [74]:
from prettytable import PrettyTable
import pandas as pd
#empty list for results
TableInfo = []
pdframe = None
#loop through each line of the table and extract information 
for line in range(TableBeg,TableEnd):
        Thisline = c[line]
        Nextline = c[line+1]
        
       # print(Thisline)
       # print(Nextline)
        #split line into words 
        split = Thisline.split('  ')
       # print(split)
        
        #remove blank elemnts from list
        split = list(filter(None, split))
        #strip white space from each element in list
        #split = [x.strip(' ') for x in split]
        
        #print(split[0])
        #if its the header line of the table split all groups (no ordering needed)
        if line ==TableBeg: 
            t = PrettyTable(split)
            TotalCols = len(split)
            TableVals = (["MISSING"],)*TotalCols
            
            #save as dataframe to allow easy manipulation of output
            pdframe = pd.DataFrame(columns = [split])
         
        if line > TableBeg and len(split)>1:
     
            #itr = len(split)
            #IF FIRST ELEMENT IN SPLIT IS A NUMERIC THEN U KNOW U FOUND A NEW LINE ENTRY(NOTE THIS WILL ONLY CAPTURE THE FIRST LINE OF THE NEW ENTRY)
            #print(itr)
            #for each column in TableVals
            
            #if  re.match(r'^[1-9][0-9]*$', stripwhite(split[0])) is not None \
            #    or  re.match(r'^[1-9][0-9]*$', stripwhite(split[1])) is not None:
            if split[0] != '': 
            #if len(stripwhite(split[0]))<2 #make this a regex condition for any number ^[1-9][0-9]*$
            
                for group in range(len(TableVals)):
                    if group == 0 and len(split)>0:  #1th element of Tablevals (min)
                        TableVals = (stripwhite(split[0]),)+TableVals[1:]
                        split = [x for x in split if x != split[0]] #remove element from split that has been acounted for

                    if group == 1  and len(split)>0:  #6th element of table vals (max)
                        TableVals = TableVals[0:TotalCols-1]+(split[len(split)-1],)
                        split = [x for x in split if x != split[len(split)-1]]

                    if group == 2  and len(split)>0:  #2nd element of table vals (min)
                        TableVals=  TableVals[:1]+(split[0],)+TableVals[2:]
                        split = [x for x in split if x != split[0]]

                    if group == 3  and len(split)>0:  #5th elment of table vals(max)
                        TableVals = TableVals[:TotalCols-2]+(split[len(split)-1],)+TableVals[len(TableVals)-1:]
                        split = [x for x in split if x != split[len(split)-1]]

                    if group == 4  and len(split)>0:  #3th elment of table vals(min)
                        TableVals=  TableVals[:2]+(split[0],)+TableVals[3:]
                        split = [x for x in split if x != split[0]]

                    if group == 5  :  #3th elment of table vals(min)
                        if len(split)==0:
                            TableVals=TableVals[:TotalCols-3]+('MISSING DATA',)+TableVals[len(TableVals)-2:]
                        else:        
                            TableVals=  TableVals[:TotalCols-3]+(split[len(split)-1],)+TableVals[len(TableVals)-2:]
                            split = [x for x in split if x != split[0]]

                t.add_row(TableVals)
                pdframe.loc[len(pdframe)+1] = TableVals

                
#rename all headers so no white space is left 
pdframe.columns = pdframe.columns.str.strip()
print(pdframe)

  Line Quantity             Item #   Description   Unit Price  Line Total
1    1   1.0000   GC - Renewal IGC  MISSING DATA   139,232.25  139,232.25


### 5.Post Processing of table information(Correct errors identified in table parsing) 

In [ ]:

#post checks 
#1.split quanity and description if quanitity has characters and numeric as first character in string 
#Rule: if description is missing(could not find multiple spaces between description and some other field)
#      and the feild before description has length > 10 (no quanitity should be this much)
#      and the first charcter in quantity is a digit \

for index,row in pdframe.iterrows():
    if  row['Description'][0] == 'MISSING'  \
    and len(row['Quantity'])>10 \
    and  re.match(r'(\s)?\d',(row['Quantity'][:2])) is not None: 

        row['Description'] = row['Quantity'][2:]
        row['Quantity']= row['Quantity'][:2]

print(pdframe)

### 4.Option2 (Leverage Static position of table information in regards to column width)

##### Note: this enables retrieval of all information for each column instead of just the rows associated with the start line for a new item(the rows with the pricing infomration) 

In [75]:
from prettytable import PrettyTable
import pandas as pd
import numpy as np

#empty list for results
TableInfo = []

pdframe = None
#loop through each line of the table and extract information 

if endvar =='Subtotal':
    
    #create empty vector for each column 
    Line = []
    Quantity=[]
    Item=[]
    Description=[]
    UnitPrice =[]
    LineTotal =[]
        
    for line in range(TableBeg,TableEnd):
        Thisline = c[line]
        Nextline = c[line+1]
       
        #print(Thisline)
            #if its the header line of the table split all groups (no ordering needed)
        if line ==TableBeg: 
                
            #split line into words 
            split = Thisline.split('  ')

            #remove blank elemnts from list
            split = list(filter(None, split))
             
            t = PrettyTable(split)
            TotalCols = len(split)
            TableVals = (["MISSING"],)*TotalCols
            
            #save as dataframe to allow easy manipulation of output
            pdframe = pd.DataFrame(columns = [split])

        if line > TableBeg and len(split)>1:
            
            Line.append((line,Thisline[:7]))
            Quantity.append((line,Thisline[8:19]))
            Item.append((line,Thisline[20:32]))
            Description.append((line,stripwhite(Thisline[33:55])))
            UnitPrice.append((line,Thisline[75:105]))
            LineTotal.append((line,Thisline[106:]))

    #rename all headers so no white space is left 
    pdframe.columns = pdframe.columns.str.strip()

    #create individual dataframes for each column(may not be needed)
    Line2 = pd.DataFrame(data = Line, columns =["Index","Line"])
    Quantity2 = pd.DataFrame(data = Quantity, columns =["Index","Quantity"])
    Item2 = pd.DataFrame(data = Item, columns =["Index","Item"])
    Description2 = pd.DataFrame(data = Description, columns =["Index","Description"])
    UnitPrice2 = pd.DataFrame(data = UnitPrice, columns =["Index","UnitPrice"])
    LineTotal2 = pd.DataFrame(data = LineTotal, columns =["Index","LineTotal"])

    #merge columns back together into final table 
    result = pd.merge(left =Line2,right = Quantity2, how="left", on="Index")
    result2 = pd.merge(left =result,right = Item2, how="left", on="Index")
    result3 = pd.merge(left =result2,right = Description2, how="left", on="Index")
    result4 = pd.merge(left =result3,right = UnitPrice2, how="left", on="Index")
    result5 = pd.merge(left =result4,right = LineTotal2, how="left", on="Index")
    print(result5)

    
if endvar =='Total':
    #create empty vector for each column 
    Item=[]
    Quantity=[]
    Description=[]
    Rate =[]
    Amount =[]
        
    for line in range(TableBeg,TableEnd):
        Thisline = c[line]
        Nextline = c[line+1]
       
        #print(Thisline)
            #if its the header line of the table split all groups (no ordering needed)
        if line ==TableBeg: 
                
            #split line into words 
            split = Thisline.split('  ')

            #remove blank elemnts from list
            split = list(filter(None, split))
             
            t = PrettyTable(split)
            TotalCols = len(split)
            TableVals = (["MISSING"],)*TotalCols
            
            #save as dataframe to allow easy manipulation of output
            pdframe = pd.DataFrame(columns = [split])

        if line > TableBeg and len(split)>1:
            
            #variable by comapny / client need
            Item.append((line,Thisline[:21]))
            Quantity.append((line,Thisline[22:36]))
            Description.append((line,stripwhite(Thisline[37:95])))
            Rate.append((line,Thisline[96:115]))
            Amount.append((line,Thisline[116:]))

    #rename all headers so no white space is left 
    pdframe.columns = pdframe.columns.str.strip()

    #create individual dataframes for each column(may not be needed)
    #Line2 = pd.DataFrame(data = Line, columns =["Index","Line"])
    Quantity2 = pd.DataFrame(data = Quantity, columns =["Index","Quantity"])
    Item2 = pd.DataFrame(data = Item, columns =["Index","Item"])
    Description2 = pd.DataFrame(data = Description, columns =["Index","Description"])
    UnitPrice2 = pd.DataFrame(data = Rate, columns =["Index","Rate"])
    LineTotal2 = pd.DataFrame(data = Amount, columns =["Index","Amount"])

    #merge columns back together into final table 
    result = pd.merge(left =Item2,right = Quantity2, how="left", on="Index")
    result2 = pd.merge(left =result,right = Description2, how="left", on="Index")
    result3 = pd.merge(left =result2,right = UnitPrice2, how="left", on="Index")
    result4 = pd.merge(left =result3,right = LineTotal2, how="left", on="Index")
    #result5 = pd.merge(left =result4,right = LineTotal2, how="left", on="Index")
    print(result4)

   Index     Line     Quantity          Item Description  \
0     15   1        1.0000      GC - Renewal         IGC   

                        UnitPrice        LineTotal  
0                      139,232.25       139,232.25  


## Functions 

In [3]:

##Function to determine if pdf is searchable 
def IsPDFSearchable(path): 
    
    #determine if pdf is searchable 
    import os, re
    
    #path is the directory with the files, other 2 are the names of the files you will store your lists in
    path = path
    
    #use this if u want to actually create a file holding all pdfs that are searchable vs not 
    #files_with_text = open("C://Users//caridza//Desktop//EY//AI//OCR//GE PO RFP//solution data//PO Examples//files_with_text.txt", 'a')
    #image_only_files = open("C://Users//caridza//Desktop//EY//AI//OCR//GE PO RFP//solution data//PO Examples//image_only_files.txt", 'a')
    
    files_with_text = []
    image_only_files = []
    
    #have os make a list of all files in that dir for a loop
    filelist = os.listdir(path)
    
    #compile regular expression that matches "Font"
    mysearch = re.compile(r'.*Font.*', re.DOTALL) #.decode("utf-8")
    
    #loop over all files in the directory, open them in binary ('rb'), search that binary for "Font"
    #if they have "Font" they have text, if not they don't
    #(pdf does something to understand the Font type and uses this word every time the pdf contains text)
    for pdf in filelist:
        openable_file = os.path.join(path, pdf)
        cat_file = open(openable_file, 'rb')
        usable_cat_file = cat_file.read()
        #print usable_cat_file
        if mysearch.match(str(usable_cat_file)):
            #files_with_text.write(pdf + '\n') #use if u are storing info in file
            files_with_text.append(pdf)
        else:
            #image_only_files.write(pdf + '\n') #use if u are storing info in file 
            image_only_files.append(pdf)
    
    #use if storing pdf names in files
    #close files holding searchable vs unsearchable pdfs
    #files_with_text.close()
    #image_only_files.close()
    return files_with_text, image_only_files


##https://www.freebsd.org/cgi/man.cgi?query=pdftotext&apropos=0&sektion=0&manpath=FreeBSD+6.2-RELEASE+and+Ports&format=html
#http://www.pdftron.com/assets/pdfs/support/PDFTron_PDF2Text_User_Manual.pdf
#options not used 
##'-remove_hidden_text',
##'-remove_invisible_text'
## pdf2text -c 150,600,250,700 license.pdf -a 1 #extract text from a speicific region 
def get_bbox_file(pdf_path, txt_path):
    if not os.path.exists(pdf_path):
        raise FileNotFoundError("Couldn't find {}".format(pdf_path))
    elif os.path.exists(txt_path):
        return txt_path
    # TXT file doesn't exist yet. Create it.
    try:
        p1 = Popen(('pdftotext','-bbox-layout',  
                       pdf_path, txt_path),
                      stdin=PIPE, stdout=PIPE)
        output = p1.communicate()
    finally:
        p1.stdout.close()
        p1.stdin.close()
    try:
        p1.terminate()
        p1.kill()
    except:
        pass

    return txt_path
    
#return tables 
#return bounding boxes
#for i in range(len(SearchablePDFPaths)):
#    bboxpath = croppath+str(SearchablePDFPaths[i].rsplit("//", 1)[1]).replace(".pdf","_BBox.txt")
#    pdfpath = SearchablePDFPaths[i]
#    bboxtextpaths.append(get_bbox_file(pdfpath, bboxpath))

#extract only bounding box information
#lines = []
#for i in range(len(bboxtextpaths)):
#    #lines.append([ line for line in open(bboxtextpaths[i]) if 'block' in line])
#    lines.append([ line.replace("</block>\n","").replace("<block","").replace(">\n","").replace(",","") for line in open(bboxtextpaths[i]) if 'block' in line])

def get_text_file(pdf_path, txt_path):
    if not os.path.exists(pdf_path):
        raise FileNotFoundError("Couldn't find {}".format(pdf_path))
    elif os.path.exists(txt_path):
        return txt_path
    # TXT file doesn't exist yet. Create it.
    try:
        p1 = Popen(('pdftotext','-layout',  
                       pdf_path, txt_path),
                      stdin=PIPE, st=PIPE)
        output = p1.communicate()
    finally:
        p1.stdout.close()
        p1.stdin.close()
    try:
        p1.terminate()
        p1.kill()
    except:
        pass

    return txt_path

#get files
def getfiles(path, extension):
    items = os.listdir(path)
    newlist = []
    for names in items:
        if names.endswith(extension):
            newlist.append(names)
    return newlist


def isPhoneNumber(text, chars = 11):
    if len(text) != chars:
        return False
    
    for i in range(0, 2):
        if not text[i].isdecimal():
            return False
        #if text[3] != '-':
        #    return False
    for i in range(3, 6):
        if not text[i].isdecimal():
            return False
        #if text[7] != '-':
        #    return False
    for i in range(chars,11):
        if not text[i].isdecimal():
            return False
    return text


def validate(email): 
    match=re.search(r"(^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9]+\.[a-zA-Z0-9.]*\.*[com|org|edu]{3}$)",email)
    if match:
        return 'Valid email.'
    else:
        return 'Invalid email.'
    

#count words in string 
def wordCount(mystring):  
    tempcount = 0  
    count = 1  
    indicator = 0  
    try:  
        for character in mystring:  
            if character == " ":  
                tempcount +=1  
                if tempcount ==1:  
                    count +=1  

                else:  
                    tempcount = 0  
        return count  

    except Exception:  
        error = "Not a string"  
        return error  

#remove all white space from string (only use to count total words in line)
def stripwhite(string):

    string = string.rstrip().lstrip() # Remove all extra spaces at the start and at the end of the string
    while "  " in string: # While  there are 2 spaces beetwen words in our string...
        string = string.replace("  ", " ") # ... replace them by one space!
    return(string)

